In [1]:
from nltk import word_tokenize
from gensim.models import KeyedVectors
import numpy as np
from pprint import pprint
from cpae.predictors import CpaeEmbedder

In [2]:
def_embeds = KeyedVectors.load_word2vec_format(
    'embeddings/separate_sense_cpae_en-camb_unfreeze-w2v.txt',
    binary=False
)

cpae = CpaeEmbedder.from_path(
    'model/separate_sense_cpae_en-camb_unfreeze-w2v/model.tar.gz',
    'cpae_embedder',
)

/home/nlplab/simon/anaconda3/envs/cpae/lib/python3.8/site-packages/allennlp/tango/__init__.py:17: UserWarning: AllenNLP Tango is an experimental API and parts of it might change or disappear every time we release a new version.
  warnings.warn(
2021-10-05 01:15:39,894 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2021-10-05 01:15:39,903 - INFO - allennlp.models.archival - loading archive file model/separate_sense_cpae_en-camb_unfreeze-w2v/model.tar.gz
2021-10-05 01:15:39,905 - INFO - allennlp.models.archival - extracting archive file model/separate_sense_cpae_en-camb_unfreeze-w2v/model.tar.gz to temp dir /tmp/tmpnzfczb23
2021-10-05 01:15:40,373 - INFO - allennlp.common.params - dataset_reader.type = dictionary
2021-10-05 01:15:40,374 - INFO - allennlp.common.params - dataset_reader.max_instances = None
2021-10-05 01:15:40,374 - INFO - allennlp.common.params - dataset_reader.manual_distributed_sharding = False
2021-10-05 01:15:40,375 - INFO - allennlp.common.params

In [3]:
# we need to tokenize the defintion before we transform it.
definition = ' '.join(word_tokenize('a round fruit with firm, white flesh and a green, red, or yellow skin'))

inputs = {
    'word': None,
    'definition': definition
}

apple_embeds = cpae.embed_inputs(inputs)

pprint(def_embeds.similar_by_vector(np.array(apple_embeds)))

2021-10-05 01:15:47,536 - INFO - gensim.models.keyedvectors - precomputing L2-norms of word weight vectors


[('apple.noun.01', 0.9999999403953552),
 ('melon.noun.01', 0.844855785369873),
 ('plum.noun.01', 0.810901403427124),
 ('pumpkin.noun.01', 0.801559329032898),
 ('cantaloupe.noun.01', 0.8008959293365479),
 ('swede.noun.01', 0.8000357151031494),
 ('rutabaga.noun.01', 0.8000357151031494),
 ('honeydew.noun.02', 0.796711802482605),
 ('peach.noun.01', 0.7909814119338989),
 ('avocado.noun.01', 0.7839112281799316)]
